In [1]:
% matplotlib inline

import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy.io as sio
from mpl_toolkits.basemap import Basemap, cm
import gsw as sw
import scipy.stats as ss

import seaborn as sns

from netCDF4 import Dataset

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Times New Roman'})

In [2]:
# calculate distance between points along each transect

def distance(data):
    transect = pd.unique(data['lon'])
    distances = pd.DataFrame()    
    for c in transect:
        temp = pd.DataFrame()
        temp = data.loc[(data['lon']==c), ['lat','lon', 'depth (m)']]
        
        # calculate distance in km between points
        temp['dx'] = np.insert(sw.distance(temp['lon'].values, temp['lat'].values, 0),0,0)
        temp['x'] = np.cumsum(temp['dx'])
        
        if c == transect[0]:
            distances = distances.append(temp)
        else:
            distances = pd.concat([distances, temp], axis=0)
        del temp
    data = pd.merge(left = data, right = distances, how='left', left_on=['lat','lon', 'depth (m)'], right_on=['lat','lon', 'depth (m)'])
    #return distances

    return data  

In [5]:
# import cruise data

in_file = '../data/KuroAlldata.csv'
data = pd.read_csv(in_file, sep = ',')

in_file = '../data/Kuro_chl_coords.csv'
chl = pd.read_csv(in_file, sep = ',')

chl = distance(chl)

print data.columns

Index([u'station', u'day/mon/yr', u'lat', u'lon', u'bottle', u'depth (m)',
       u'T(C)', u'S', u'sigma_t', u'theta', u'DOC (mg C/l)', u'DOC (uM)',
       u'TDN (mg N/l)', u'TDN (uM)', u'SiO2 (mg SiO2/l)', u'SiO2 (uM)',
       u'DON (mg N/l)', u'DON (uM)', u'NO3 (ug N/l)', u'NO3 (uM)',
       u'NH4 (ug N/l)', u'NH4(uM)', u'TDP (ug P/l)', u'TDP (uM)',
       u'PO4 (ug P/l)', u'PO4 (uM)', u'DOP (ug P/l)', u'DOP (uM)', u'P*',
       u'Si*', u'Pro (cells/l)', u'Pro (mg Cm^3l)', u'Syn (cells/l)',
       u'Syn (mg C/m3)', u'Picoeuks (cells/l)', u'Diatoms (cells/l)',
       u'Dinoflagellates (cells/l)', u'Haptophytes (cells/l)',
       u'Diatom Diversity', u'Dino Diversity', u'Total Diversity',
       u'[TChl a] (ug/l)', u'[TChl b] (ug/l)', u'[TChl c] (ug/l)',
       u'[Caro] (ug/l)', u'[But fuco] (ug/l)', u'[Hex fuco] (ug/l)',
       u'[Allo] (ug/l)', u'[Diad] (ug/l)', u'[Diato] (ug/l)', u'[Fuco] (ug/l)',
       u'[Perid] (ug/l)', u'[Zea] (ug/l)', u'[Chl a] (ug/l)',
       u'[DVChl a] (ug/l

In [4]:
# do chl calibration

sub = chl[['fluorescence', 'chlorophyll (ug/l)']].dropna()

mT,cT, rT, pT, std_errT = ss.linregress(sub['fluorescence'],sub['chlorophyll (ug/l)'])
chl['CHL'] = mT*chl['fluorescence']+cT

print 'estimated calibration slope = ', mT, ', r = ', rT, ', p-value < 0.05 ', pT<0.05

#integrate fluorescence and T, S data
sub = data[['station','bottle', 'depth (m)', 'T(C)', 'theta', 'S', 'sigma_t']]

temp = chl.merge(sub, on = ['station','bottle', 'depth (m)'], how = 'inner')
temp = temp.drop(['chlorophyll (ug/l)', 'Unnamed: 7'], axis = 1)

temp['sample'] = temp["station"].map(str) + '-' + temp["bottle"].map(str)
temp['theta'] = sw.conversions.pt_from_t(temp['S'], temp['T(C)'], temp['depth (m)'])
temp = temp[temp['station']<41]

temp.to_csv('../data/data_CHL.csv', sep = ',')

estimated calibration slope =  1.24413568902 , r =  0.937132630412 , p-value < 0.05  True


/Users/sclayton/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:809: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)
/Users/sclayton/anaconda/lib/python2.7/site-packages/gsw-3.0.3-py2.7.egg/gsw/utilities/utilities.py:251: RuntimeWarning: invalid value encountered in less
  SA[SA < 0] = np.ma.masked
/Users/sclayton/anaconda/lib/python2.7/site-packages/gsw-3.0.3-py2.7.egg/gsw/gibbs/library.py:899: RuntimeWarning: invalid value encountered in greater
  nonzero_SA = np.any(SA > 0)
